In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
# ニフティ不動産　賃貸　大阪府江坂駅
url = 'https://myhome.nifty.com/rent/osaka/esaka_st/?lines=osaka:osakashimidosujisen'

result = requests.get(url)
c = result.content
soup = BeautifulSoup(c, "html.parser")

In [14]:
# ページ数を取得
body = soup.find("body notLoggedIn")
pages = body.find_all("div", {'class':'column'})
pages_text = str(pages)
pages_split = pages_text.split('</a></li>\n</ol>')
num_pages = int(pages_split[0].split('>')[-1])
print("ページ数＝",num_pages)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
# 全てのページのURLを作成
# urlを入れる箱（リスト）を設定
urls = []

# 1ページ目を格納
urls.append(url)

# 2ページ目以降を格納
for i in range(num_pages-1):
    pg = str(i+2)
    url_page = url + '&pn=' + pg
    urls.append(url_page)

NameError: name 'num_pages' is not defined

In [132]:
data = []
errors = []
for url in urls:
    summary = soup.find("div",{'id':'js-bukkenList'})
    cassetteitems = summary.find_all("div",{'class':'cassetteitem'})
    for cas in cassetteitems:
            # 情報取得用の箱を準備します。
            subtitle = '' # 物件名
            location = '' # 住所
            station = ''   # 最寄駅（リスト）
            yrs = ''     # 築年数
            level = ''    # 階数
            value = ''  # 家賃
            value2 = '' # 管理費・共益費
            floor = '' # 間取り
            area = 0 # 面積
            
            # 物件名
            subtitle = cas.find_all("div", "cassetteitem_content-title")[0].string
            # 住所
            location = cas.find_all("li", "cassetteitem_detail-col1")[0].string
            # 最寄駅
            station = cas.find_all("div", "cassetteitem_detail-text")[0].string
            # 築年数
            yrs = cas.find_all("li", "cassetteitem_detail-col3")[0].select("div")[0].string
            # 階数
            level = cas.find_all("li", "cassetteitem_detail-col3")[0].select("div")[1].string
            # 家賃
            value = cas.find_all("span", "cassetteitem_price cassetteitem_price--rent")[0].string
            # 管理費・共益費
            value2 = cas.find_all("span", "cassetteitem_price cassetteitem_price--administration")[0].string
            # 間取り
            floor = cas.find_all("span", "cassetteitem_madori")[0].string
            # 面積
            area = cas.find_all("span", "cassetteitem_menseki")[0].text
    
            data.append([subtitle, location, station, yrs, level, value, value2, floor, area])
            

In [133]:
# data listを DataFrameに変換
df = pd.DataFrame(data, columns=['物件名','住所','最寄り駅','築年数', '階数', '家賃', '管理費・共益費', '間取り', '面積'])

In [134]:
# csvファイルとして保存
df.to_csv('suumo_bukken.csv', sep = ',',encoding='utf-8-sig')

In [ ]:
#山﨑テスト

In [ ]:
#山﨑テスト２